## Data

Спочатку завантажуємо word embeddings для української мови.

In [1]:
!curl http://lang.org.ua/static/downloads/models/news.lowercased.tokenized.word2vec.300d.bz2 --output news.lowercased.tokenized.word2vec.300d.bz2

In [2]:
!bunzip2 news.lowercased.tokenized.word2vec.300d.bz2

In [3]:
from gensim.models import KeyedVectors

%time wv = KeyedVectors.load_word2vec_format('news.lowercased.tokenized.word2vec.300d', binary=False)

CPU times: user 55.9 s, sys: 563 ms, total: 56.5 s
Wall time: 56.3 s


In [4]:
wv.most_similar('слово')

[('дієслово', 0.6502863764762878),
 ('слівце', 0.6484909653663635),
 ('словосполучення', 0.6456568241119385),
 ('гасло', 0.5913079977035522),
 ('слово**', 0.555127739906311),
 ("прислів'я", 0.5407627820968628),
 ('письмо', 0.5235773324966431),
 ('прізвище', 0.52119380235672),
 ('пророцтво', 0.5125285983085632),
 ('ремесло', 0.5058826804161072)]

Потім розпаковуємо та завантажуємо дані.

In [5]:
!unzip -q ../../../tasks/1551.zip

In [6]:
!ls 1551 | head -n 15

Аварійний--травмонебезпечний-стан-утримання-об-єктів-благоустрою.txt
Бажаючі-отримати--Картки-киянина--КК--.txt
Будівництво-АЗС.txt
Будівництво-в-нічний-час.txt
Будівництво-дооблаштування-дитячого-майданчику.txt
Будівництво--дооблаштування-спортивних-майданчиків.txt
Будівництво-та-реконструкція-об-єктів-освіти.txt
Взаємовідносини-з-сусідами.txt
Вивезення--утилізація-твердих-та-негабаритних-відходів.txt
Видалення-аварійних--пошкоджених-хворобами-дерев.txt
Видача-розрахункових-книжок--квитанцій--довідок.txt
Вилов-безпритульних-тварин.txt
Вирізування--кронування--гілля-дерев.txt
Виток-холодної-води-на-поверхню.txt
Відновлення-благоустрою-після-вик--планових-аварійних-робіт-на-об-єктах-благоуст.txt
ls: write error: Broken pipe


In [7]:
import glob

files = glob.glob("1551/*")

files[:10]

['1551/Незадовільна-температура-ГВП.txt',
 '1551/Незадовільне-обслуговування-в-амбулаторно-поліклінічних-установах.txt',
 '1551/Відсутнє-електропостачання.txt',
 '1551/Порушення-правил-тиші--після-------.txt',
 '1551/Неякісне-ХВП.txt',
 '1551/Нанесення-дорожньої-розмітки.txt',
 '1551/Робота-циркуляційної-системи.txt',
 '1551/Встановлення-світлофора.txt',
 '1551/Завезення-піску-на-дитячий-майданчик.txt',
 '1551/Скошування-трави.txt']

In [8]:
import os
from collections import namedtuple

Document = namedtuple('Document', 'id topic_id tags content')

def parse_tags(file):
    return [tag for tag in os.path.basename(file.name)[:-4].split('-') if tag]

def parse_topic_file(topic_id, filename):
    documents = []    
    with open(filename) as f:
        tags = parse_tags(f)        
        _id = None
        content = []
        for line in f:            
            if _id is None and line.strip().isnumeric():
                _id = int(line.strip())                
                continue
            if not (_id is None) and line.strip():                
                content.append(line.strip())
                continue
            if not (_id is None) and not line.strip():
                documents.append(Document(_id, topic_id, tags, ''.join(content)))
                _id = None
                content = []                
    
    return documents

In [9]:
all_documents = [doc for topic_id, file in enumerate(files) \
                 for doc in parse_topic_file(topic_id, file) if len(doc.content) > 0]

len(all_documents)

114338

In [10]:
all_documents[:3]

[Document(id=2697865, topic_id=0, tags=['Незадовільна', 'температура', 'ГВП'], content='Недогрев горячей воды (вода нормальной температуры подавалась неделю с15 по 23, до этого была частичная подача горячей воды (пару часов вечером и ночью горячая), остальное время теплой), сейчас опять температура порядка 40 градусов. Эта ситуация продолжается на фоне постоянного недогрева батарей в квартире, ДУ 12 ККЕУ  МОУкраины  не реагирует на ситуацию.'),
 Document(id=3170827, topic_id=0, tags=['Незадовільна', 'температура', 'ГВП'], content='Из горячего крана течет холодная вода, в вечернее и утреннее время купаться нет возможности. Необходимо или пересчитывать тарифы или включать горячую воду.'),
 Document(id=3165270, topic_id=0, tags=['Незадовільна', 'температура', 'ГВП'], content='Я поживаю на 6 этаже 9и - этажного дома на протяжении долгого промежутка времени у нас в квартире из крана горячей воды, особенно утром и в первой половине дня течёт если не холодная вода, то вода чуть тёплая. По утр

Тепер фільтруємо документи з українською мовою.

In [11]:
from langdetect import detect
from langdetect.detector import LangDetectException
from tqdm import tqdm_notebook

def is_uk(text):
    
    if len(text):
        try:
            return detect(text[:1024]) == 'uk'
        except LangDetectException as e:
            return False
    
    return False

uk_documents = [doc for doc in tqdm_notebook(all_documents) if is_uk(doc.content)]

In [12]:
uk_documents[:3]

[Document(id=3152784, topic_id=0, tags=['Незадовільна', 'температура', 'ГВП'], content='Відсутнітність горячого водопостачання належної температури в ванній кімнаті та на кухні. Було звернення в ЖКХ та  на горячу лінію 1551. Досі питання не вирішено'),
 Document(id=3143050, topic_id=0, tags=['Незадовільна', 'температура', 'ГВП'], content='Добрий вечір.Прошу розібратися з проблемою невідповідної температури гарячої води, прийняти міри та відновити гаряче водопостачання.Вже 3 дні гаряча вода має температру не більше 38 градусів.При неможливості відновити постачання гарчоїї води, прошу Вас перерахувати її вартість згідно з законодавством.Дякую за допомогу та розуміння.'),
 Document(id=3142427, topic_id=0, tags=['Незадовільна', 'температура', 'ГВП'], content='На моє звернення № Г-6623 відповідь написав директор КП РЕО-1 Кононець В. В. Під час зустрічі активу нашого будинку із начальником управління ЖКГ Святошинського району Мужиченком Є. О. пан Мужиченко сказав, що згідно Закону України № 

Дивимся на дані.

In [13]:
import pandas

uk_doc_df = pandas.DataFrame([doc._replace(tags = "-".join(doc.tags)) for doc in uk_documents])

In [14]:
uk_doc_df.head(10)

,id,topic_id,tags,content
0,3152784,0,Незадовільна-температура-ГВП,Відсутнітність горячого водопостачання належно...
1,3143050,0,Незадовільна-температура-ГВП,Добрий вечір.Прошу розібратися з проблемою нев...
2,3142427,0,Незадовільна-температура-ГВП,На моє звернення № Г-6623 відповідь написав ди...
3,3130991,0,Незадовільна-температура-ГВП,Доброго дня! Вже більше двох тижнів гаряче вод...
4,2405990,0,Незадовільна-температура-ГВП,На звернення:Номер звернення:\tГ-6478Зареєстро...
5,3115494,0,Незадовільна-температура-ГВП,"Звертаюсь до Вас стосовно вирішення питання, щ..."
6,3104107,0,Незадовільна-температура-ГВП,Доброго дня!!! Моє звернення від 02.12.14 #Г-1...
7,3091571,0,Незадовільна-температура-ГВП,Протягом останнього тижня гаряча вода недостат...
8,2690156,0,Незадовільна-температура-ГВП,Прошу прийняти необхідні заходи по покращенню ...
9,2748419,0,Незадовільна-температура-ГВП,немає температури гарячої води


In [15]:
uk_doc_df.describe(include='all')

,id,topic_id,tags,content
count,6.148600e+04,61486.000000,61486,61486
unique,NaN,NaN,188,53743
top,NaN,NaN,Відсутність-ГВП,Доброго дня!
freq,NaN,NaN,6531,707
mean,3.038086e+06,105.549361,NaN,NaN
std,2.625866e+05,55.921234,NaN,NaN
min,1.000000e+01,0.000000,NaN,NaN
25%,2.840977e+06,58.000000,NaN,NaN
50%,3.083505e+06,121.000000,NaN,NaN
75%,3.245302e+06,150.000000,NaN,NaN


In [16]:
uk_doc_df.groupby(['topic_id', 'tags']).count().sort_values(['id'], ascending = False).head(50)

,,id,content
topic_id,tags,,
138,Відсутність-ГВП,6531,6531
180,Укладання-та-ремонт-асфальтного-покриття,3618,3618
27,Відсутність-опалення,3128,3128
127,Перевірка-дозвільної-документації-демонтаж-кіосків-ларків,2196,2196
67,Прибирання-та-санітарний-стан-територій,2004,2004
79,Технічний-стан-проїжджих-частин-вулиць-та-тротуарів,1292,1292
155,Відновлення-благоустрою-після-вик-планових-аварійних-робіт-на-об-єктах-благоуст,1276,1276
121,Відсутність-освітлення-у-під-їзді-за-відсутності-несправності-лампочок,1260,1260
58,Не-працює-пасажирський-ліфт,1216,1216


Виділяємо лейбли.

In [17]:
import numpy as np

topic_labels = np.array([doc.topic_id for doc in uk_documents])
len(topic_labels)

61486

І розбиваємо дані на тренувальні і тестові.

In [18]:
from sklearn.model_selection import train_test_split

train_documents, test_documents, train_topic_labels, test_topic_labels = \
    train_test_split(uk_documents, topic_labels, random_state = 26, test_size = 0.3)

In [19]:
print(len(train_documents))
print(len(train_topic_labels))

43040
43040


In [20]:
print(len(test_documents))
print(len(test_topic_labels))

18446
18446


## Baseline

Будуємо бейзлайн, знаходимо суму векторів слів по кожному документу і використовуємо kNN на знайденних векторах. Для порівняння векторів застосовуємо cosine similarity. Перед знаходженням суми векторів, документ токенізується та видаляються stop words. Знайдені вектори нормалізуються, в такому випадку eclidean distance для kNN має той самий ефект що й cosine distance, при цьому алгоритм дозволяє використовувати більш ефективні структури данних, такі як, наприклад, k-d tree.

In [21]:
import tokenize_uk
import string

with open('uk_stop_words.txt') as f:
    STOP_WORDS = f.read().split()
    
EXT_PUNCTUATION = "”..."

def non_stop_word(word):
    return not (word in string.punctuation or word in EXT_PUNCTUATION \
                or word in STOP_WORDS or word.isnumeric()) and word.isalpha() and len(word) > 3

def remove_stop_words(tokens):
    return [token for token in tokens if non_stop_word(token.lower())]

def tokenize_doc(doc):
    return [word.lower() for word in \
            remove_stop_words(tokenize_uk.tokenize_words(doc.content))]

def normalize_vec(x):
    m = np.max(x)
    if m > 0.0:
        return x/np.sqrt(np.dot(x,x))
    return x
    
def doc_to_sum_vec(doc):
    words = tokenize_doc(doc)    
    vec = np.zeros(300)
    for word in words:
        try:
            vec += wv[word]
        except KeyError as e:            
            pass
        
    return vec

Рахуємо вектори для тренувальних і тестових документів.

In [22]:
train_doc_sum_vecs = np.array([doc_to_sum_vec(doc) for doc in tqdm_notebook(train_documents)])

In [23]:
test_doc_sum_vecs = np.array([doc_to_sum_vec(doc) for doc in tqdm_notebook(test_documents)]) 

Тренуємо модель.

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

class Model:
    def __init__(self, train_vectors, train_labels, test_vectors, test_labels):
        self.train_vectors = train_vectors
        self.train_labels = train_labels
        self.test_vectors = test_vectors
        self.test_labels = test_labels
        
    def train(self):
        self.model.fit(self.train_vectors, self.train_labels)
        self.topics_predicted = self.model.predict(self.test_vectors)
        
    def test(self):
        print(classification_report(self.test_labels, self.topics_predicted))  

class KnnModel(Model):
    def __init__(self, train_vectors, train_labels, test_vectors, test_labels, n = 1):
        super().__init__(np.array([normalize_vec(doc) for doc in train_vectors]),\
                       train_labels,\
                       np.array([normalize_vec(doc) for doc in test_vectors]),\
                       test_labels)                
        self.model = KNeighborsClassifier(n_neighbors = n, algorithm='kd_tree', metric = 'euclidean', n_jobs = 6)    

In [25]:
knn = KnnModel(train_doc_sum_vecs, train_topic_labels, test_doc_sum_vecs, test_topic_labels)

In [26]:
%time knn.train()

CPU times: user 10min 44s, sys: 114 ms, total: 10min 44s
Wall time: 1min 53s


In [27]:
knn.test()

              precision    recall  f1-score   support

           0       0.22      0.50      0.31       335
           1       0.70      0.65      0.67        43
           2       0.40      0.27      0.33        77
           3       0.42      0.24      0.31        46
           4       0.27      0.16      0.20        25
           5       0.44      0.38      0.41       113
           6       0.21      0.30      0.25       142
           7       0.24      0.32      0.27        50
           8       0.64      0.55      0.59        29
           9       0.31      0.16      0.21        25
          10       0.54      0.47      0.50       159
          11       0.25      0.18      0.21        34
          12       0.54      0.33      0.41        21
          13       0.13      0.18      0.15        33
          14       0.25      0.22      0.23        79
          15       0.25      0.31      0.28        13
          16       0.55      0.60      0.58       127
          17       0.17    

## Imrovements

Намагаємося покращити результат. Спочатку будемо використовувати логістичну регресію, потім проробимо все те саме але з векторами Doc2Vec.

In [28]:
from sklearn.linear_model import LogisticRegression

class LogregModel(Model):
    def __init__(self, train_vectors, train_labels, test_vectors, test_labels, iters = 3000):
        super().__init__(train_vectors, train_labels, test_vectors, test_labels)
        self.model = LogisticRegression(random_state=26, n_jobs = 6, solver="lbfgs", \
                                        multi_class="multinomial", max_iter = iters)

In [29]:
logreg = LogregModel(train_doc_sum_vecs, train_topic_labels, test_doc_sum_vecs, test_topic_labels)

In [30]:
%time logreg.train()

CPU times: user 375 ms, sys: 557 ms, total: 932 ms
Wall time: 39min 13s


In [31]:
logreg.test()

              precision    recall  f1-score   support

           0       0.68      0.51      0.58       335
           1       0.87      0.63      0.73        43
           2       0.46      0.61      0.52        77
           3       0.36      0.28      0.32        46
           4       0.50      0.40      0.44        25
           5       0.47      0.44      0.46       113
           6       0.30      0.20      0.24       142
           7       0.20      0.24      0.22        50
           8       0.66      0.72      0.69        29
           9       0.52      0.44      0.48        25
          10       0.63      0.59      0.61       159
          11       0.10      0.12      0.11        34
          12       0.56      0.43      0.49        21
          13       0.15      0.24      0.18        33
          14       0.20      0.16      0.18        79
          15       0.32      0.46      0.37        13
          16       0.52      0.63      0.57       127
          17       0.27    

Є невелике покращення в якості. Переходимо до Doc2Vec. Для цього використовуємо gensim. Спочатку конвертуємо наші документи в модель gensim.

In [32]:
from gensim.models.doc2vec import TaggedDocument

def to_tagged_doc(doc):
    words = tokenize_doc(doc)
    return TaggedDocument(words, [doc.topic_id])

In [33]:
to_tagged_doc(uk_documents[15])

TaggedDocument(words=['заявляю', 'чергове', 'втручання', 'діяльність', 'інформаційної', 'системи', 'колцентру', 'метою', 'викривлення', 'відомостей', 'стосовно', 'результатів', 'вирішення', 'поданих', 'звернень', 'даний', 'відмітку', 'виконано', 'наводжу', 'витяг', 'березня', 'перегляд', 'інтерактивній', 'картівідсутність', 'відповідальний', 'печерська', 'брама', 'мазурчак', 'олександр', 'володимирович', 'дата', 'контролю', 'березня', 'статус', 'виконано', 'відповідаю', 'дійсності', 'оскільки', 'годин', 'офіційно', 'отриманий', 'електронний', 'запит', 'виклав', 'текст', 'протилежного', 'змісту', 'наводжу', 'заявника', 'квітня', 'статус', 'виконано', 'квартира', 'розташована', 'відношення', 'перекриття', 'стояка', 'заміна', 'вентиля', 'сусідів', 'потребує', 'перекриття', 'водопостачання', 'будинку', 'викличе', 'появу', 'трубах', 'будинку', 'стільки', 'бруду', 'зливався', 'понад', 'хвилини', 'хвилину', 'вигаданих', 'нормативів', 'скарги', 'взагалі', 'подавались', 'заради', 'отримання', '

In [34]:
train_tagged_docs = [to_tagged_doc(doc) for doc in tqdm_notebook(train_documents)]

In [35]:
test_tagged_docs = [to_tagged_doc(doc) for doc in tqdm_notebook(test_documents)]

Потім тренуємо PV-DBOW модель. Розмір вектору документа 300, як і в моделі з embeddins, яку ми використовували в бейзлайні.

In [36]:
from gensim.models.doc2vec import Doc2Vec

dbow_model = Doc2Vec(dm=0, vector_size=300, min_count=5, window=10, workers=6, epochs=120)

dbow_model.build_vocab(train_tagged_docs + test_tagged_docs)

In [37]:
%time dbow_model.train(train_tagged_docs, total_examples=dbow_model.corpus_count, epochs=dbow_model.epochs)

CPU times: user 10min 12s, sys: 47.5 s, total: 10min 59s
Wall time: 4min 23s


Збираємо вектори документів.

In [38]:
train_doc_vecs = np.array([dbow_model.infer_vector(doc.words) for doc in tqdm_notebook(train_tagged_docs)])

In [39]:
test_doc_vecs = np.array([dbow_model.infer_vector(doc.words) for doc in tqdm_notebook(test_tagged_docs)])

Знову намагаємося застосувати kNN та логістичну регресію на отриманних векторах.

In [40]:
knn2 = KnnModel(train_doc_vecs, train_topic_labels, test_doc_vecs, test_topic_labels)

In [41]:
%time knn2.train()

CPU times: user 12min 16s, sys: 237 ms, total: 12min 16s
Wall time: 2min 10s


In [42]:
knn2.test()

              precision    recall  f1-score   support

           0       0.55      0.56      0.55       335
           1       0.77      0.77      0.77        43
           2       0.45      0.52      0.48        77
           3       0.41      0.39      0.40        46
           4       0.61      0.56      0.58        25
           5       0.51      0.62      0.56       113
           6       0.36      0.30      0.32       142
           7       0.38      0.56      0.45        50
           8       0.48      0.86      0.62        29
           9       0.65      0.68      0.67        25
          10       0.73      0.72      0.73       159
          11       0.38      0.15      0.21        34
          12       0.56      0.43      0.49        21
          13       0.39      0.27      0.32        33
          14       0.34      0.28      0.31        79
          15       0.26      0.38      0.31        13
          16       0.53      0.70      0.60       127
          17       0.21    

In [43]:
logreg2 = LogregModel(train_doc_vecs, train_topic_labels, test_doc_vecs, test_topic_labels)

In [44]:
%time logreg2.train()

CPU times: user 293 ms, sys: 536 ms, total: 828 ms
Wall time: 5min 48s


In [45]:
logreg2.test()

              precision    recall  f1-score   support

           0       0.56      0.52      0.54       335
           1       0.91      0.70      0.79        43
           2       0.51      0.57      0.54        77
           3       0.55      0.37      0.44        46
           4       0.78      0.56      0.65        25
           5       0.64      0.52      0.58       113
           6       0.33      0.30      0.31       142
           7       0.50      0.60      0.55        50
           8       0.62      0.72      0.67        29
           9       0.65      0.60      0.63        25
          10       0.84      0.72      0.77       159
          11       0.50      0.12      0.19        34
          12       0.80      0.38      0.52        21
          13       0.47      0.24      0.32        33
          14       0.32      0.25      0.28        79
          15       0.54      0.54      0.54        13
          16       0.52      0.71      0.60       127
          17       0.33    

## Conslusion

Бачимо що вдалося покращити якість у порівнянні з бейзланойм більш ніж на 10% згідно F1. Логістична регресія в порівнянні з kNN у всіх випадках працювала краще. Вектори документів також дали покращення у всіх випадках.